In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path

from sdm_eurec4a.visulization import set_custom_rcParams, label_from_attrs
from sdm_eurec4a.reductions import mean_and_stderror_of_mean

set_custom_rcParams()

import matplotlib.pyplot as plt

In [ ]:
data_dir = Path(
    "/home/m/m301096/CLEO/data/debug_output/long_duration_128/collision_condensation/combined/eulerian_dataset_combined.nc"
)
ds = xr.open_dataset(data_dir)
ds["radius_bins"].attrs.update(long_name="Radius", units="µm")

In [ ]:
data = (
    1e3
    * (ds["mass_represented"] / ds["gridbox_volume"])
    .mean(dim="time", keep_attrs=True, skipna=True)
    .compute()
)


fig, axs = plt.subplots(nrows=3, figsize=(8, 6))

for _ax in axs[:2]:
    _ax.plot(
        data.sum("radius_bins").T,
        ds["gridbox_coord3"].T,
    )

    _ax.set_ylabel("Height [m]")
    _ax.set_xlabel("Mass concentration [g/m³]")

axs[1].set_xscale("log")

axs[2].hist(
    1e2 * (data.sel(gridbox=1) / data.sel(gridbox=ds["max_gridbox"])).values.flatten(),
    bins=np.arange(0, 200, 10),
)

fig.tight_layout()

In [ ]:
data = 1e3 * (ds["mass_difference"] / ds["gridbox_volume"]).mean(dim="time", keep_attrs=True).compute()


fig, axs = plt.subplots(nrows=2, figsize=(8, 6))

for _ax in axs:
    _ax.plot(
        data.sum("radius_bins").T,
        ds["gridbox_coord3"].T,
    )

    _ax.set_ylabel("Height [m]")
    _ax.set_xlabel("Mass concentration [g/m³]")

axs[1].set_xscale("log")

fig.tight_layout()

plot the cloud and subcloud layer PSD and MSD

In [ ]:
ds_select = ds.sel(gridbox=ds["max_gridbox"])
psd = ds_select["xi"] / ds_select["gridbox_volume"]
psd.attrs.update(
    {
        "long_name": "Particle size distribution",
        "units": "m^{-3}",
        "standard_name": "particle_size_distribution",
    }
)

msd = 1e3 * (ds_select["mass_represented"] / ds_select["gridbox_volume"])
msd.attrs.update(
    {
        "long_name": "Mass size distribution",
        "units": "g m^{-3}",
        "standard_name": "mass_size_distribution",
    }
)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 6))

psd_m, psd_sem = mean_and_stderror_of_mean(psd, dims=("time",))

axs[0].plot(
    psd["radius_bins"],
    psd_m.T,
    alpha=0.5,
)
axs[0].set_yscale("log")
axs[0].set_ylabel(label_from_attrs(psd))

axs[1].plot(msd["radius_bins"], msd.mean(dim="time").T)
axs[1].set_ylabel(label_from_attrs(msd))

for _ax in axs:
    _ax.set_xlabel(label_from_attrs(ds_select["radius_bins"]))
    _ax.set_xscale("log")

In [ ]:
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite
from sdm_eurec4a import RepositoryPath
from tqdm import tqdm

RP = RepositoryPath("levante")
repo_dir = RP.repo_dir
data_dir = RP.data_dir

ds_clouds = xr.open_dataset(
    repo_dir
    / "data/observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
)
ds_clouds = ds_clouds.swap_dims({"time": "cloud_id"})
ds_cloud_composite = xr.open_dataset(
    repo_dir / "data/observation/cloud_composite/processed/cloud_composite_SI_units_20241025.nc"
)

In [ ]:
lwc_obs_mean_list = []
lwc_obs_sem_list = []
nbc_obs_mean_list = []
nbc_obs_sem_list = []
psd_obs_mean_list = []
psd_obs_sem_list = []

for cloud_id in tqdm(ds["cloud_id"]):
    ds_select = match_clouds_and_cloudcomposite(
        ds_clouds=ds_clouds.sel(cloud_id=cloud_id),
        ds_cloudcomposite=ds_cloud_composite.sel(radius=slice(52e-6, 3e-3)),
    )
    m, s = mean_and_stderror_of_mean(ds_select["liquid_water_content"], dims=("time",))
    m, s = m.expand_dims(dim=dict(cloud_id=[cloud_id])), s.expand_dims(dim=dict(cloud_id=[cloud_id]))
    lwc_obs_mean_list.append(m)
    lwc_obs_sem_list.append(s)

    nbc = ds_select["particle_size_distribution_non_normalized"].sum("radius", keep_attrs=True)
    m, s = mean_and_stderror_of_mean(nbc, dims=("time",))
    m, s = m.expand_dims(dim=dict(cloud_id=[cloud_id])), s.expand_dims(dim=dict(cloud_id=[cloud_id]))

    nbc_obs_mean_list.append(m)
    nbc_obs_sem_list.append(s)

    psd = ds_select["particle_size_distribution_non_normalized"]
    m, s = mean_and_stderror_of_mean(psd, dims=("time",))
    m, s = m.expand_dims(dim=dict(cloud_id=[cloud_id])), s.expand_dims(dim=dict(cloud_id=[cloud_id]))

    psd_obs_mean_list.append(m)
    psd_obs_sem_list.append(s)


lwc_obs_mean = xr.concat(lwc_obs_mean_list, dim="cloud_id")
lwc_obs_sem = xr.concat(lwc_obs_sem_list, dim="cloud_id")
nbc_obs_mean = xr.concat(nbc_obs_mean_list, dim="cloud_id")
nbc_obs_sem = xr.concat(nbc_obs_sem_list, dim="cloud_id")
psd_obs_mean = xr.concat(psd_obs_mean_list, dim="cloud_id")
psd_obs_sem = xr.concat(psd_obs_sem_list, dim="cloud_id")

100%|██████████| 128/128 [00:06<00:00, 21.16it/s]


In [ ]:
ds_select = ds.sel(gridbox=ds["max_gridbox"]).sel(radius_bins=slice(52, 3e3))

lwc = 1e3 * ds_select["mass_represented"].sum("radius_bins") / ds_select["gridbox_volume"]
lwc_cleo_mean, lwc_cleo_sem = mean_and_stderror_of_mean(lwc, dims=("time",))

nbc = ds_select["xi"].sum("radius_bins") / ds_select["gridbox_volume"]
nbc_cleo_mean, nbc_cleo_sem = mean_and_stderror_of_mean(nbc, dims=("time",))

psd = ds_select["xi"] / ds_select["gridbox_volume"]
psd_cleo_mean, psd_cleo_sem = mean_and_stderror_of_mean(psd, dims=("time",))

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 6))

axs[0].errorbar(
    x=nbc_obs_mean,
    xerr=2 * nbc_obs_sem,
    y=nbc_cleo_mean,
    yerr=2 * nbc_cleo_sem,
    linestyle="",
    marker=".",
)

axs[1].errorbar(
    x=lwc_obs_mean,
    xerr=2 * lwc_obs_sem,
    y=lwc_cleo_mean,
    yerr=2 * lwc_cleo_sem,
    linestyle="",
    marker=".",
)

<ErrorbarContainer object of 3 artists>

In [ ]:
import matplotlib.colors as mcolors